In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("data/Leads.csv")
df1 = pd.read_excel("data/Leads Data Dictionary.xlsx")
df1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,NaN,Variables,Description
2,NaN,Prospect ID,A unique ID with which the customer is identif...
3,NaN,Lead Number,A lead number assigned to each lead procured.
4,NaN,Lead Origin,The origin identifier with which the customer ...
